# Shapash Library

**Shapash** by *MAIF* é um kit de ferramentas Python que facilita a compreensão de modelos de aprendizado de máquina para cientistas de dados. Isso torna mais fácil compartilhar e discutir a interpretabilidade do modelo com não especialistas em dados: analistas de negócios, gerentes e usuários finais.

Concretamente, o Shapash fornece visualizações fáceis de ler e um aplicativo da web. O Shapash exibe os resultados com o texto apropriado (pré-processamento inverso / pós-processamento). O Shapash é útil em um contexto operacional, pois permite que os cientistas de dados usem a explicabilidade da exploração à produção: você pode facilmente implantar a explicabilidade local na produção para completar cada uma de suas previsões / recomendações com um resumo da explicabilidade local.

Documentação: https://shapash.readthedocs.io/en/latest/<br>
Artigo: https://www.kdnuggets.com/2021/04/shapash-machine-learning-models-understandable.html

In [ ]:
# instalando a biblioteca
#!pip install shapash

In [ ]:
# importando as bilbiotecas
import pandas as pd
from shapash.data.data_loader import data_loading

from category_encoders import OrdinalEncoder

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from shapash.explainer.smart_explainer import SmartExplainer

import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

Para testar, vamos utilizar no conjunto de dados “Preços das casas” do Kaggle para ajustar um regressor e prever os preços das casas.

In [ ]:
# importando o conjunto de dados
house_df, house_dict = data_loading('house_prices')

# checando as primeiras linhas
house_df.head()

In [ ]:
# checando as dimensões
house_df.shape

In [ ]:
# definindo X
X = house_df.drop(['SalePrice'], axis=1)

# definindo y
y = house_df['SalePrice'].to_frame()

# criando uma lista com nomes das variáveis categóricas
categorical_features = house_df.select_dtypes('object').columns.tolist()

# instanciando o Ordinal Encoder
encoder = OrdinalEncoder(cols=categorical_features)

# treinando o Ordinal Encoder
encoder.fit(X)

# transformando as variáveis
X = encoder.transform(X)

# checando as primeiras linhas transformadas
X.head(2)

In [ ]:
# Separando os dados em treino e teste
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3)

# intanciando modelo
reg = RandomForestRegressor(n_estimators=200, min_samples_leaf=2)

# treinando o modelo
reg.fit(Xtrain,ytrain)

# realizando as previsões
y_pred = pd.DataFrame(reg.predict(Xtest), columns=['pred'], index=Xtest.index)

## Começando analisar com SmartExplainer Object

A classe SmartExplainer é o objeto principal da biblioteca Shapash. Ele permite que os cientistas de dados realizem várias operações para tornar os resultados mais compreensíveis: vincular codificadores, modelos, previsões, ditado de rótulo e conjuntos de dados. Os usuários do SmartExplainer têm vários métodos descritos a seguir.

In [ ]:
# instanciando o SmartExplainer como objeto
xpl = SmartExplainer(features_dict=house_dict) # parametro opcional que especifica cada nome da coluna

In [ ]:
# compilando
xpl.compile(
    x=Xtest,
    model=reg,
    preprocessing=encoder,# Optional: use inverse_transform method
    y_pred=y_pred # Optional
)

In [ ]:
# iniciando o webapp
app = xpl.run_app()

In [ ]:
# "derruba" o webapp
app.kill()